Ejercicio
Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

In [9]:
!curl https://raw.githubusercontent.com/freddy120/nlp_ceia_2022/main/tp3/courtney_barnett.txt -o courtney_barnett.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 44422  100 44422    0     0   699k      0 --:--:-- --:--:-- --:--:--  699k


In [11]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('courtney_barnett.txt', sep='/n', header=None)
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0
0,In the morning I’m slow. I drag a chair over t...
1,Well time is money and money is no man’s frien...
2,The day begins to shine. The parent teaches th...
3,Well time is money and money is no man’s frien...
4,Next door the kids run amok. The mother scream...


In [12]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 852


### Preprocesamiento

In [13]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [39]:
# Demos un vistazo
sentence_tokens[582]

['i',
 'am',
 'normally',
 'pretty',
 'forgiving',
 'but',
 'only',
 'if',
 'you',
 'are']

### Crear los vectores (word2vec)

In [19]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [40]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=3,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [41]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [42]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 852


In [43]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 473


### Entrenar el modelo

In [44]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 107510.53125
Loss after epoch 1: 44296.578125
Loss after epoch 2: 39439.765625
Loss after epoch 3: 39792.328125
Loss after epoch 4: 38186.765625
Loss after epoch 5: 38758.875
Loss after epoch 6: 38315.78125
Loss after epoch 7: 39474.59375
Loss after epoch 8: 38238.46875
Loss after epoch 9: 38995.03125
Loss after epoch 10: 38519.5625
Loss after epoch 11: 39795.84375
Loss after epoch 12: 39410.875
Loss after epoch 13: 39157.1875
Loss after epoch 14: 39762.25
Loss after epoch 15: 38777.625
Loss after epoch 16: 38667.0625
Loss after epoch 17: 39707.8125
Loss after epoch 18: 39556.25
Loss after epoch 19: 39803.375


(85397, 169480)

In [45]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["pretty"], topn=10)

[('low', 0.9997239112854004),
 ('dead', 0.9996819496154785),
 ('normally', 0.9996780157089233),
 ('that’s', 0.9996739625930786),
 ('any', 0.9996697306632996),
 ('joke', 0.9996662735939026),
 ('half', 0.9996654987335205),
 ('stories', 0.9996640682220459),
 ('easy', 0.9996618032455444),
 ('true', 0.9996607303619385)]

In [46]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["pretty"], topn=10)

[('eye', -0.6947838068008423),
 ('an', -0.7308681607246399),
 ('for', -0.8778183460235596),
 ('go', -0.9154656529426575),
 ('hear', -0.9598406553268433),
 ('let', -0.9631259441375732),
 ('gotta', -0.9723544120788574),
 ('from', -0.9777893424034119),
 ('tonight', -0.9793435335159302),
 ('don’t', -0.9816494584083557)]

### Visualizar agrupación de vectores

In [47]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [48]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
